# COMP5318 - Machine Learning and Data Mining: Assignment 1

In [1]:
import pandas as pd
import os
print(os.listdir("./Input/train"))
pd.set_option('display.max_columns', 10)

from IPython.display import set_matplotlib_formats, display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import graphviz

%matplotlib inline

['train.csv']


In [2]:
# train.csv including feature and label using for training model.
data_train_df = pd.read_csv('./Input/train/train.csv') 

In [3]:
# Selecting input feature
data_train_feature = data_train_df.loc[:, "v1":"v784"].to_numpy()

# Selecting output lable 
data_train_label = data_train_df.label.to_numpy()

In [4]:
# import matplotlib.pyplot as plt
# data_train_feature = data_train_feature.reshape((data_train_feature.shape[0], 28, 28))
# plt.imshow(data_train_feature[0], cmap=plt.get_cmap('gray'))
# plt.title("class " + str(data_train_label[0]) + ": Pullover" )
# plt.show()

# data_train_df.loc[:, "v1":"v784"]

## DATA PRE-PROCESSING FOR TRAINING DATA

In [5]:
data_train_feature = data_train_feature.reshape(30000, 28*28)
data_train_feature.shape

(30000, 784)

In [6]:
# Data Normalisation
data_train_feature = pd.DataFrame(data_train_feature)
skewed_feats = pd.DataFrame(data_train_feature).apply(lambda x: skew(x.dropna()))
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

data_train_feature[skewed_feats] = np.log1p(data_train_feature[skewed_feats])

In [7]:
# Filling NAs with the mean of the columns
data_train_feature = data_train_feature.fillna(data_train_feature.mean())
data_train_feature 

,0,1,2,3,4,...,779,780,781,782,783
0,0.0,0.0,0.0,0.000000,0.693147,...,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.693147,0.000000,...,4.127134,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
29995,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0
29996,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0
29997,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0
29998,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0


In [8]:
# Data Standardisation
data_train_feature = StandardScaler().fit_transform(data_train_feature)

In [9]:
pca = PCA(n_components=0.9)
principalComponents = pca.fit_transform(data_train_feature)

principalDf = pd.DataFrame(data = principalComponents)

In [10]:
finalDf = pd.concat([principalDf, data_train_df.label], axis = 1)

## DATA PRE-PROCESSING FOR TESTING DATA

In [11]:
# test_input.csv includes 5000 samples used for label prediction. Test samples do not have labels.
data_test_df = pd.read_csv('./Input/test/test_input.csv')
data_test_df

,id,v1,v2,v3,v4,...,v780,v781,v782,v783,v784
0,0,0,0,0,0,...,0,0,0,0,0
1,1,0,0,0,0,...,0,0,0,0,0
2,2,0,0,0,0,...,0,0,0,0,0
3,3,0,0,0,0,...,0,0,0,0,0
4,4,0,0,0,0,...,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,0,0,0,0,...,0,0,0,0,0
4996,4996,0,0,0,0,...,0,0,0,0,0
4997,4997,0,0,0,0,...,0,0,0,0,0
4998,4998,0,0,0,0,...,0,0,0,0,0


In [12]:
# Data Normalisation
skewed_feats = data_test_df.apply(lambda x: skew(x.dropna()))
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

data_test_df[skewed_feats] = np.log1p(data_test_df[skewed_feats])

In [13]:
# Filling NAs with the mean of the columns
data_test_df = data_test_df.fillna(data_test_df.mean())
data_test_df

,id,v1,v2,v3,v4,...,v780,v781,v782,v783,v784
0,0,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0
1,1,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0
2,2,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0
3,3,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0
4,4,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0
4996,4996,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0
4997,4997,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0
4998,4998,0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0


In [14]:
# Data Standardisation
data_test_df = StandardScaler().fit_transform(data_test_df)

In [15]:
# Dimension Reduction
pca = PCA(n_components=140) # Based on 90% explain variance for the training data 
principalComponents_test = pca.fit_transform(data_test_df)
principalDf_test = pd.DataFrame(data = principalComponents_test)

## KNN

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    principalDf.to_numpy(), data_train_label, random_state=0)

In [17]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (22500, 140)
y_train shape: (22500,)


In [18]:
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_test shape: (7500, 140)
y_test shape: (7500,)


In [19]:
knn = KNeighborsClassifier(n_neighbors=2)

In [20]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=2)

In [21]:
y_pred = knn.predict(X_test)
y_pred

array([5, 3, 2, ..., 9, 1, 8])

In [22]:
knn.score(X_test, y_test)

0.8396

In [ ]:
# Parameter Tuning
param_grid = {'n_neighbors': [1, 3, 5, 11, 15], 'p': [1, 2]}

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, return_train_score=True)
grid_search.fit(X_train, y_train)

print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

In [ ]:
predictions = []

for i in principalDf_test.to_numpy():
    prediction = knn.predict([list(i)])
    predictions.append(prediction[0])

In [ ]:
finaldf_test_knn_initial = pd.DataFrame(predictions)
finaldf_test_knn_initial.insert(0, 'id', range(0,len(finaldf_test_knn_initial)))
finaldf_test_knn_initial.columns = ['id', 'label']
finaldf_test_knn_initial

In [ ]:
finaldf_test_knn_initial.to_csv('knnTestInitialPredictions.csv', index = False)

## LOGISTIC REGRESSION

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(principalDf.to_numpy(), data_train_label, 
                                                    stratify=data_train_label, random_state=42)

In [ ]:
logreg = LogisticRegression(max_iter = 5000)
logreg.fit(X_train, y_train)
print("Accuracy on trainig set:", logreg.score(X_train, y_train)*100)
print("Accuracy on test set:", logreg.score(X_test, y_test)*100)

## NAIVE BAYES

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(principalDf.to_numpy(), data_train_label, 
                                                    stratify=data_train_label, random_state=42)

nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
print("Accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

## DECISION TREE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(principalDf.to_numpy(), data_train_label, 
                                                    stratify=data_train_label, random_state=42)

tree = DecisionTreeClassifier(criterion='entropy', max_depth=12, random_state=42)
tree.fit(X_train, y_train)

In [ ]:
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))